In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus

embeddings = OpenAIEmbeddings()

In [4]:
from milvus import default_server

In [8]:
default_server.start()

In [9]:
docs = [
    # jurassic park
    Document(page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose", 
             metadata={"year": 1993, "rating": 7.7, "genre": "action"}),
    # toy story
    Document(page_content="Toys come alive and have a blast doing so",
             metadata={"year": 1995, "genre": "animated", "rating": 9.3 }),
    # finding nemo
    Document(page_content="A dad teams up with a disabled partner to break into a dentist\'s office to save his son.",
             metadata={"year": 2003, "genre": "animated", "rating": 8.2 }),
    # unbearable weight of massive talent
    Document(page_content="Nicholas Cage plays Nicholas Cage in this movie about Nicholas Cage.",
             metadata={"year": 2022, "genre": "comedy", "rating": 7.0 }),
    # lord of war
    Document(page_content="Nicholas Cage sells guns until he has enough money to marry his favorite model. Then he sells more guns.",
             metadata={"year": 2005, "genre": "comedy", "rating": 7.6 }),
    # ghost rider
    Document(page_content="Nicholas Cage loses his skin and sets his skull on fire. Then he rides a motorcycle.",
             metadata={"year": 2007, "genre": "action", "rating": 5.3 }),
]

vector_store = Milvus.from_documents(
    docs,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": default_server.listen_port},
    collection_name="movies"
)

In [10]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vector_store, document_content_description, metadata_field_info, verbose=True
)

In [11]:
# This example only specifies a relevant query
retriever.get_relevant_documents("What are some movies about Nicholas Cage?")

/Users/yujiantang/Documents/workspace/hello_world_project/hw_milvus/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='Nicholas Cage' filter=None limit=None


[Document(page_content='Nicholas Cage plays Nicholas Cage in this movie about Nicholas Cage.', metadata={'year': 2022, 'rating': 7.0, 'genre': 'comedy'}),
 Document(page_content='Nicholas Cage loses his skin and sets his skull on fire. Then he rides a motorcycle.', metadata={'year': 1995, 'rating': 5.3, 'genre': 'action'}),
 Document(page_content='Nicholas Cage sells guns until he has enough money to marry his favorite model. Then he sells more guns.', metadata={'year': 2005, 'rating': 7.6, 'genre': 'comedy'}),
 Document(page_content="A dad teams up with a mentally disabled partner to break into a dentist's office to save his son.", metadata={'year': 2003, 'rating': 8.2, 'genre': 'animated'})]